# Oversizing analysis

Notion results: https://www.notion.so/bib-batteries/Autres-Result-analysis-2852de3b75c780398206d037ec530a0a?source=copy_link#29c2de3b75c780dda161d280f20a7d88

In [ ]:
import plotly.express as px
from core.sql_utils import *
from core.gsheet_utils import *
from activation.config.mappings import mapping_vehicle_type
from activation.config.mappings import mapping_vehicle_type

import pandas as pd
import numpy as np

## Data

In [ ]:
engine = get_sqlalchemy_engine()
df_dbeaver = pd.read_sql("""SELECT vm.id, vm.model_name, vm.type, vm.version, vm.autonomy, b.battery_chemistry, b.capacity, b.net_capacity, m.make_name, o.oem_name FROM vehicle_model vm
join battery b on b.id=vm.battery_id
join make m on m.id=vm.make_id
join oem o on o.id=vm.oem_id""", engine)
df_dbeaver['oversizing_ratio'] = df_dbeaver['net_capacity'] / df_dbeaver['capacity']
df_dbeaver['oversizing_kwh'] = abs(df_dbeaver['net_capacity'] - df_dbeaver['capacity'])
#df_dbeaver['size_scaled'] = 1 - ((df_dbeaver['oversizing_ratio'] - df_dbeaver['oversizing_ratio'].min()) / (df_dbeaver['oversizing_ratio'].max() - df_dbeaver['oversizing_ratio'].min())) + 0.01
df_dbeaver['size_scaled'] = (1 - ((np.log(df_dbeaver['oversizing_ratio']) - np.log(df_dbeaver['oversizing_ratio']).min()) / (np.log(df_dbeaver['oversizing_ratio']).max() - np.log(df_dbeaver['oversizing_ratio']).min()))) * 40 + 8


In [ ]:
df_scrapping = load_excel_data("Courbes de tendance", "Courbes OS")
df_scrapping = pd.DataFrame(columns=df_scrapping[:1][0], data=df_scrapping[1:])
df_scrapping = df_scrapping.rename(columns={"OEM": "make_name", "Odomètre (km)": "odometer", "Année": "year", "SoH": "soh", "Modèle": "model_name"})
df_scrapping['model_name'] = df_scrapping['model_name'].apply(lambda x: str.lower(x))
def safe_mapping_vehicle_type(row):
    model_name = row['model_name']
    if not model_name or model_name == "unknown":
        return np.nan
    try:
        return mapping_vehicle_type(row['Type'], row['make_name'], model_name, df_dbeaver)
    except Exception as e:
        # pour tracer les erreurs sans planter
        print(f"⚠️ Erreur sur {row['make_name']} {model_name}: {e}")
        return np.nan

df_scrapping['type_2'] = df_scrapping.apply(safe_mapping_vehicle_type, axis=1)

In [ ]:
df_info = (df_scrapping.merge(df_dbeaver, how='left', left_on=['type_2'], right_on=['id'])
           [["make_name_x", "model_name_y", "autonomy", "battery_chemistry", "capacity", "net_capacity", "soh", "odometer", "year", "price", "oversizing_ratio", "oversizing_kwh", "size_scaled"]]
           .rename(columns={"make_name_x": "make", "model_name_y": "model"}))

In [ ]:
df_info['price'] = df_info['price'].replace('', np.nan)

In [ ]:
df_info['price']

In [ ]:
df_info[["price",  "autonomy", "oversizing_ratio", "oversizing_kwh"]] = df_info[["price", "autonomy", "oversizing_ratio", "oversizing_kwh"]].astype(float)

In [ ]:
df_info.dropna(subset=['make', 'model', 'capacity', 'net_capacity'], inplace=True)

In [ ]:
# 90% sur le marché
model_list = ['#1', 
 '2',
 '4',
 '500e',
 '600e',
 'ariya',
 'atto 3',
 'avenger',
 'born',
 'bz4x',
 'e-c4',
 "ë-c4",
 'corsa-e',
 'corsa electric',
 'countryman',
 'e-2008',
 'e-208',
 'e-3008',
 'e:ny1',
 'ec40',
 'enyaq',
 'enyaq coupe',
 'eqa',
 'eqb',
 'eqe',
 'eqe suv',
 'ev6',
 'ev9',
 'ex30',
 'ex40',
 'e-expert combi',
 'fortwo',
 'i4',
 'i5',
 'id. buzz',
 'id. buzz lwb',
 'id. buzz nwb',
 'id.3',
 'id.4',
 'id.5',
 'id.7',
 'ioniq 5',
 'ioniq 6',
 'ix',
 'ix1',
 'ix2',
 'ix3',
 'kangoo',
 'kona',
 "kona electric",
 'leaf',
 'megane',
 'model 3',
 'model y',
 "mokka",
 'mokka-e',
 "mokka electric",
 'mustang mach-e',
 'mx-30',
 'e-niro',
 'one/cooper',
 'partner',
 'q4 e-tron',
 'q4 sportback e-tron',
 'q8 e-tron',
 'q8 e-tron sportback',
 'q8 sportback e-tron'
 'scenic',
 'spring',
 'taycan',
 'twingo',
 'vito',
 'zoe',
 'zs']

In [ ]:
mask = df_dbeaver["model_name"].isin(model_list)
df_dbeaver_filtered = df_dbeaver[mask].copy()

In [ ]:
# Classification des modèles par classe
def classify_vehicle(model_name, make_name=None):
    """
    Classifie un modèle de véhicule en citadine, berline, SUV ou utilitaire
    """
    model_lower = str(model_name).lower()
    
    # Utilitaires
    utilitaires = ['kangoo', 'partner', 'e-expert combi', 'vito', 'id. buzz', 'id. buzz lwb', 'id. buzz nwb']
    if any(util in model_lower for util in utilitaires):
        return 'Utilitaire'
    
    # SUV
    suv_keywords = ['suv', 'countryman', 'q8', 'q4', 'eqe suv', 'enyaq', 'enyaq coupe', 
                    'eqa', 'eqb', 'ix', 'ix1', 'ix2', 'ix3', 'ev9', 'ex30', 'ex40', 
                    'ec40', 'id.4', 'id.5', 'bz4x', 'e-3008', 'model y', 'mustang mach-e',
                    'ariya', 'ev6', 'e:ny1', 'e-2008']
    if any(keyword in model_lower for keyword in suv_keywords):
        return 'SUV'
    
    # Citadines
    citadines = ['zoe', 'spring', 'twingo', 'fortwo', '500e', 'e-208', 'corsa-e', 
                 'corsa electric', 'mokka', 'mokka-e', 'mokka electric', 'e-c4', 'ë-c4',
                 'id.3', 'born', 'one/cooper', 'i3', 'leaf', 'kona', 'kona electric',
                 'e-niro', 'mx-30', 'zs', 'atto 3', 'avenger', '2', '4', '#1']
    if any(cit in model_lower for cit in citadines):
        return 'Citadine'
    
    # Berlines
    berlines = ['model 3', 'i4', 'i5', 'eqe', 'id.7', 'ioniq 5', 'ioniq 6', 'taycan',
                'megane', 'scenic', 'e-3008', '600e', 'q4 sportback e-tron', 
                'q8 e-tron sportback', 'q8 sportback e-tron']
    if any(ber in model_lower for ber in berlines):
        return 'Berline'
    

df_dbeaver_filtered['classe'] = df_dbeaver_filtered['model_name'].apply(classify_vehicle)


## Oversizing VS €

In [ ]:
df_filtered = df_info[df_info["model"].isin(df_info.value_counts("model").head(20).index)]

# Nuage de points
fig = px.scatter(
    df_filtered,
    x="oversizing_kwh",
    y="price",
    color="model",
    opacity=0.5,
    # size="size_scaled",
    # size_max=20,
    title="Relationship between oversizing (kWh) and price",
    labels={"oversizing_kwh": "Oversizing (kWh)", "price": "Price (€)", "size_scaled": "Oversizing ratio"},
    
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.update_traces(marker=dict(size=10))  
fig.show()

In [ ]:
# fig.write_html("../graphs/oversizing_vs_price.html")


## Oversizing VS autonomy

In [ ]:

fig = px.scatter(
    df_dbeaver_filtered,
    x="oversizing_kwh",
    y="autonomy",
    color="model_name",
    # size="size_scaled",
    # size_max=20,
    title="Relationship between oversizing (kWh) and autonomy by model",
    labels={"oversizing_kwh": "Oversizing (kWh)", "autonomy": "Autonomy (km)", "model_name": "Model", "size_scaled": "Oversizing ratio"},
    opacity=0.7
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.update_traces(marker=dict(size=10))  

fig.show()

In [ ]:
# fig.write_html("../graphs/oversizing_vs_autonomy_model.html")


In [ ]:
df_dbeaver[df_dbeaver["battery_chemistry"]!= "UNKNOWN"]

In [ ]:

fig = px.scatter(
    df_dbeaver[df_dbeaver["battery_chemistry"]!= "UNKNOWN"],
    x="oversizing_kwh",
    y="autonomy",
    color="battery_chemistry",
    # size="size_scaled",
    # size_max=20,
    title="Relationship between oversizing (kWh) and autonomy by battery chemistry",
    labels={"oversizing_kwh": "Oversizing (kWh)", "autonomy": "Autonomy (km)", "battery_chemistry": "Battery chemistry", "size_scaled": "Oversizing ratio"},
    color_discrete_map={
        'LFP': '#007046',
        'NCA': '#0f0f0f',
        'NMC': '#00f095'
    },
    opacity=0.5
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.update_traces(marker=dict(size=10))  
fig.show()

In [ ]:
# fig.write_html("../graphs/oversizing_vs_autonomy_chemistry.html")

In [ ]:
df_dbeaver.groupby("battery_chemistry").agg({"oversizing_kwh": ["mean","median", "max", "min"], "autonomy": ["mean","median", "max", "min"]})


In [ ]:

fig = px.scatter(
    df_dbeaver_filtered,
    x="oversizing_kwh",
    y="autonomy",
    color="classe",
    # size="size_scaled",
    # size_max=20,
    title="Relationship between oversizing (kWh) and autonomy by vehicle category",
    labels={"oversizing_kwh": "Oversizing (kWh)", "autonomy": "Autonomy (km)", "classe": "Car category", "size_scaled": "Oversizing ratio"},
    color_discrete_map={
        'Citadine': '#007046',
        'Berline': '#0f0f0f',
        'SUV': '#00f095',
        'Utilitaire': '#FF7276'
    },
    opacity=0.5
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.update_traces(marker=dict(size=10))  

fig.show()

In [ ]:
# fig.write_html("../graphs/oversizing_vs_autonomy_car_category.html")


## Oversizin by OEM/Make

In [ ]:
oem_list = ['tesla', 'byd',  'bmw', 'volvo-cars', 'stellantis', 'hyundai', 'volkswagen', 'mercedes-benz',  'renault', 
            'toyota', 'nissan', 'kia', 'ford', 'skoda', 'volvo', 'mazda', 'honda']

df_dbeaver_oem = df_dbeaver[df_dbeaver['oem_name'].isin(oem_list)]

In [ ]:
df_agg = (
    df_dbeaver_oem.groupby("oem_name")
    .agg(
        max=("oversizing_kwh", "max"),
        min=("oversizing_kwh", "min"),
        mean=("oversizing_kwh", "mean"),
        median=("oversizing_kwh", "median")
    )
    .reset_index()
    .sort_values("mean")
)
custom_colors = [
    "#1b9e77",
    "#00f095",
    "#8ec5a0",
    "#005c73",
    "#88eecc",
    "#007046"
]

fig = px.bar(
    df_agg,
    x="oem_name",
    y=["mean"],
    barmode="group",
    title="Average oversizing (kWh) by OEM",
    labels={"value": "kWh", "oem_name": "OEM"},
    color_discrete_sequence=["#007046"]
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.show()

In [ ]:
fig.write_html("../graphs/oversizing_by_oem_kwh.html")

In [ ]:
df_agg = (
    df_dbeaver_filtered.groupby("oem_name")
    .agg(
        max=("oversizing_ratio", "max"),
        min=("oversizing_ratio", "min"),
        mean=("oversizing_ratio", "mean"),
        median=("oversizing_ratio", "median")
    )
    .reset_index()
    .sort_values("mean")
)

fig = px.bar(
    df_agg,
    x="oem_name",
    y=["mean"],
    barmode="group",
    title="Average oversizing ratio net/gross by OEM",
    labels={"value": "ratio net/gross", "oem_name": "OEM"}
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.show()

In [ ]:
# fig.write_html("../graphs/oversizing_ratio_by_oem.html")


In [ ]:
df_agg = (
    df_dbeaver_filtered.groupby("make_name")
    .agg(
        max=("oversizing_ratio", "max"),
        min=("oversizing_ratio", "min"),
        mean=("oversizing_ratio", "mean"),
        median=("oversizing_ratio", "median")
    )
    .reset_index()
    .sort_values("mean")
)

fig = px.bar(
    df_agg,
    x="make_name",
    y=["mean"],
    barmode="group",
    title="Average oversizing ratio net/gross by brand",
    labels={"value": "ratio net/gross", "make_name": "Brand"}
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.show()

In [ ]:
# fig.write_html("../graphs/oversizing_ratio_by_make.html")


In [ ]:
df_agg = (
    df_dbeaver_filtered.groupby("make_name")
    .agg(
        max=("oversizing_kwh", "max"),
        min=("oversizing_kwh", "min"),
        mean=("oversizing_kwh", "mean"),
        median=("oversizing_kwh", "median")
    )
    .reset_index()
    .sort_values("mean")
)

fig = px.bar(
    df_agg,
    x="make_name",
    y=["mean"],
    barmode="group",
    title="Average oversizing (kWh) by brand",
    labels={"value": "kWh", "make_name": "Brand"}
)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.update_xaxes(showgrid=True, gridcolor='grey')
fig.update_yaxes(showgrid=True, gridcolor='grey')
fig.show()

In [ ]:
# fig.write_html("../graphs/oversizing_by_make_kwh.html")
